<a href="https://colab.research.google.com/github/goldlight1004/AI_NLP/blob/main/%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 사전을 이용한 감성분석

In [ ]:
#from google.colab import auth
#auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# 형태소분석기 관련 설치
!apt-get update
!apt-get install g++ openjdk-8-jdk

!pip install JPype1
!pip install rhinoMorph

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic/main Sources [1,867

In [ ]:
# 경로 변경
%cd /content/gdrive/My\ Drive/pytest/

/content/gdrive/My Drive/pytest


In [ ]:
!ls

네이버뉴스-생활문화_다중  kor-eng	       ratings_small.txt  윤동주시.txt
aclImdb_v1_small	  negative.txt	       similarity	  wiki_test.txt
fra-eng			  positive.txt	       test.csv		  word2vec
iris.csv		  ratings_morphed.txt  김소월시.txt


In [ ]:
# 데이터 로딩
def read_data(filename, encoding='cp949'):                # 읽기 함수 정의
  with open(filename, 'r', encoding=encoding) as f:
    data = [line.split('\t') for line in f.read().splitlines()]
    data = data[1:]                 # txt 파일의 헤더(id document label)는 제외하기
  return data

def write_data(data, filename, encoding='cp949'):         # 쓰기 함수도 정의
  with open(filename, 'w', encoding=encoding) as f:
    f.write(data)

#data = read_data('/content/gdrive/My Drive/pytest/ratings_small.txt', encoding='cp949')
data = read_data('ratings.txt', encoding='cp949')         # 전체파일은 ratings.txt (긍정 1만, 부정 1만)

In [ ]:
print(len(data))

200000


In [ ]:
# 전체 데이터 형태소 분석
import rhinoMorph
rn = rhinoMorph.startRhino()

morphed_data = ''
for data_each in data:
  morphed_data_each = rhinoMorph.onlyMorph_list(rn, data_each[1], pos=['NNG', 'NNP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ'])
  joined_data_each = ' '.join(morphed_data_each)			        # 문자열을 하나로 연결
  if joined_data_each:                                      	# 내용이 있는 경우만 저장하게 함
    morphed_data += data_each[0]+"\t"+joined_data_each+"\t"+data_each[2]+"\n"
    
# 형태소 분석된 파일 저장
write_data(morphed_data, 'ratings_morphed.txt', encoding='cp949')

filepath:  /usr/local/lib/python3.6/dist-packages
classpath:  /usr/local/lib/python3.6/dist-packages/rhinoMorph/lib/rhino.jar
RHINO started!


In [ ]:
# 형태소 분석된 데이터 로딩
data = read_data('ratings_morphed.txt', encoding='cp949')

print(len(data))   
print(len(data[0])) 
print(data[0])

197513
3
['8132799', '디자인 배우 학생 외국 디자이너 일구 전통 통하 발전 문화 산업 부럽 사실 우리나라 그 어렵 시절 끝 열정 지키 노라노 같 전통 있 같 사람 꿈 꾸 이루 나가 있 감사', '1']


In [ ]:
# 감정사전 읽기
data_id = [line[0] for line in data]       		# 데이터 id
data_text = [line[1] for line in data]     		# 데이터 본문
data_senti = [line[2] for line in data]    		# 데이터 긍부정 부분

positive = read_data('positive.txt')		      # 긍정 감정사전 읽기
negative = read_data('negative.txt')		      # 부정 감정사전 읽기

print("positive:", positive)
print("negatvie:", negative)
                     
pos_found = []                                # 각 문장에서 발견될 긍정어의 개수 
neg_found = []                                # 각 문장에서 발견될 부정어의 개수 

positive: [['가능'], ['가능하'], ['가락'], ['가치'], ['간단하'], ['간단히'], ['간직하'], ['간호'], ['간호하'], ['감동'], ['감사'], ['감사하'], ['감싸'], ['감탄하'], ['강화'], ['강화되'], ['강화하'], ['갚'], ['개방되'], ['개방하'], ['개선'], ['개선되'], ['개선하'], ['개혁'], ['거침없이'], ['건강'], ['건장하'], ['걸맞'], ['격려'], ['결심'], ['결심하'], ['결혼'], ['결혼하'], ['경감'], ['경감되'], ['공언'], ['공언하'], ['공정하'], ['관대하'], ['관리되'], ['굉장하'], ['구원'], ['굳'], ['귀엽'], ['귀하'], ['균형'], ['그치'], ['극복'], ['극복하'], ['근절되'], ['근절하'], ['긁'], ['긍정'], ['기념'], ['기대'], ['기부하'], ['기쁘'], ['기쁨'], ['기사도'], ['기적'], ['기회'], ['깨끗하'], ['꼼꼼하'], ['꽃'], ['꿀'], ['끌리'], ['나아가'], ['낙'], ['낫'], ['넓'], ['넓히'], ['노력'], ['노력하'], ['논리'], ['논의'], ['논의하'], ['놀랍'], ['놀이'], ['늘리'], ['다이아몬드'], ['다정'], ['다행히'], ['단장'], ['달래'], ['달콤하'], ['대단'], ['대담하'], ['대화'], ['덕'], ['도덕'], ['도전'], ['독립'], ['독립되'], ['돌보'], ['돕'], ['동의'], ['동의하'], ['되살리'], ['드라이브'], ['따르'], ['뚜렷하'], ['뛰어나'], ['띄우'], ['마법'], ['만족'], ['만족하'], ['맑'], ['맑음'], ['맛'], ['맛있'], ['맞'], ['맞추'], ['매력'], ['멋지'], ['메이저'], ['면밀히'], ['명백히'], ['명시되'], ['명예']

In [ ]:
# 감정단어 파악

def cntWordInLine(data, senti):
  senti_found = []
  for onedata in data:
    oneline_word = onedata.split(' ')       # 한 줄의 데이터를 공백 단위로 분리하여 리스트로 저장
    senti_temp = 0
    for sentiword in senti:      
      if sentiword[0] in oneline_word:      # posword[0] 하여 리스트를 문자열로 추출
        senti_temp += 1                     # 현재의 감정단어와 일치하면 숫자를 하나 올려 줌 (중복은 계산 안 함)
    senti_found.append(senti_temp)          # 현재의 줄에서 찾은 감성단어의 숫자를 해당 위치에 저장
  return senti_found
      
      
data_senti_poscnt = cntWordInLine(data_text, positive)      # 발견된 긍정 단어의 숫자 파악
data_senti_negcnt = cntWordInLine(data_text, negative)      # 발견된 부정 단어의 숫자 파악

print(data_senti_poscnt)
print(data_senti_negcnt)

[5, 1, 0, 0, 2, 1, 0, 0, 0, 1, 1, 1, 0, 1, 2, 0, 1, 0, 1, 0, 3, 0, 0, 0, 3, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 2, 2, 2, 0, 1, 1, 1, 0, 1, 0, 3, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 0, 0, 2, 1, 0, 0, 0, 2, 2, 2, 2, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 2, 0, 0, 1, 1, 1, 2, 4, 0, 1, 0, 0, 0, 1, 3, 1, 2, 0, 0, 1, 2, 1, 0, 2, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 2, 0, 0, 0, 3, 0, 0, 0, 1, 1, 0, 3, 0, 1, 1, 1, 3, 1, 0, 4, 4, 1, 1, 3, 0, 1, 1, 0, 1, 0, 3, 2, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 2, 2, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 2, 0, 1, 3, 0, 2, 0, 1, 1, 0, 3, 1, 3, 0, 0, 0, 0, 1, 0, 1, 2, 0, 0, 2, 1, 1, 2, 1, 0, 1, 0, 0, 2, 1, 0, 1, 0, 1, 0, 1, 2, 0, 1, 1, 1, 1, 0, 2, 1, 1, 1, 1, 0, 0, 0, 0, 0, 2, 2, 0, 2, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 4, 0, 1, 1, 0, 0, 0, 0, 3, 0, 0, 0, 0, 1, 1, 0, 0, 1, 2, 2, 2, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 2, 0, 1, 0, 2, 0, 0, 1, 1, 1, 1, 2, 0, 0, 1, 1, 4, 2, 2, 0, 2, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 

In [ ]:
# 감정점수 계산
# Pandas 데이터프레임으로 저장
import pandas as pd
newdata = pd.DataFrame({'id':data_id, 'text':data_text, 'original':data_senti, 
                        'pos':data_senti_poscnt, 'neg':data_senti_negcnt})
senti_score = newdata['pos'] - newdata['neg']      # 긍정개수에서 부정개수를 뺌
newdata['senti_score'] = senti_score               # 그 수를 senti_score 컬럼에 저장 

newdata.loc[newdata.senti_score > 0, 'new'] = 1    # 새로운 긍부정 기호
newdata.loc[newdata.senti_score <= 0, 'new'] = 0   # 새로운 긍부정 기호

# 처음에 기록된 긍부정과 새로 계산된 긍부정이 같은지 여부를 matched 컬럼에 저장
# original 컬럼은 문자로 되어 있으므로 숫자로 변환 뒤 비교 
newdata.loc[pd.to_numeric(newdata.original) == newdata.new, 'matched'] = 'True'
newdata.loc[pd.to_numeric(newdata.original) != newdata.new, 'matched'] = 'False'

In [ ]:
newdata.head()

In [ ]:
# 원점수와 비교 및 저장
score = newdata.matched.str.count('True').sum() / (newdata.matched.str.count('True').sum() + newdata.matched.str.count('False').sum()) * 100
print(score)

newdata.to_csv('newfile.csv', sep=',', encoding='cp949', index=False) 	# csv 저장
newdata.to_csv('newfile2.txt', sep='\t', encoding='cp949', index=False) 	# 또는 txt 저장

In [ ]:
# 시그모이드 점수 계산
import math
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

newdata['sigmoid'] = newdata.senti_score.apply(sigmoid)

In [ ]:
newdata.head()